# 4. Optimizing and profiling the model

TODO: Add description

In [1]:
import timeit
import pandas as pd
import modelx as mx

In [2]:
ex1 = mx.read_model('CashValue_ME_EX1') 

In [3]:
timeit.timeit('ex1.Projection.result_pv()', globals=globals(), number=1)

1.2499775000000004

In [4]:
ex1.clear_all()

In [5]:
ex4 = mx.read_model('CashValue_ME_EX4') 

In [6]:
timeit.timeit('ex4.Projection.result_pv()', globals=globals(), number=1)

0.2862044000000008

In [7]:
ex4.clear_all()

TODO: Add description

## Testing with 100,000 scenarios

TODO: Add description

<div class="alert alert-warning">

**Warning:**

takes about 4GB of memory space 
    
</div>

In [8]:
ex1.Projection.scen_size = 100_000

In [9]:
timeit.timeit('ex1.Projection.result_pv()', globals=globals(), number=1)

8.0889741

In [10]:
ex1.clear_all()

In [11]:
ex4.Projection.scen_size = 100_000

In [12]:
timeit.timeit('ex4.Projection.result_pv()', globals=globals(), number=1)

2.3929682999999997

In [13]:
ex4.clear_all()

## Profile the runs

TODO: Add description

In [14]:
def get_time_info(model):
    
    try:
        mx.start_stacktrace(maxlen=None)
        model.Projection.result_pv()
        df = pd.DataFrame.from_dict(
            mx.get_stacktrace(summarize=True), orient="index")
    finally:
        mx.stop_stacktrace()
        model.clear_all()

    # Remove model and space names from index
    prefixlen = len(model.name + '.Projection.')
    df.index = [s[prefixlen:] for s in df.index]     

    # Add duration %
    total = df['duration'].sum()
    df['dur_perc'] = df['duration'] * 100 / total
    
    return df.sort_values(['dur_perc'], ascending=False)

In [15]:
ex1.Projection.scen_size = 10_000
df1 = get_time_info(ex1)
df1[['duration', 'dur_perc']].iloc[:10]

,duration,dur_perc
surr_charge_rate(t),0.697034,39.238824
inv_return_mth(t),0.338032,19.029207
premium_pp(t),0.146663,8.256274
pols_new_biz(t),0.069975,3.939195
inv_income_pp(t),0.069556,3.915600
"claims(t, kind)",0.064405,3.625641
expenses(t),0.035827,2.016841
pols_maturity(t),0.032865,1.850092
"claim_pp(t, kind)",0.032804,1.846696
prem_to_av_pp(t),0.031291,1.761523


In [16]:
ex4.Projection.scen_size = 10_000
df4 = get_time_info(ex4)
df4[['duration', 'dur_perc']].iloc[:10]

,duration,dur_perc
premium_pp(t),0.033878,10.744304
mort_rate_mth(t),0.032263,10.231875
surr_charge_rate(t),0.024496,7.768857
duration(t),0.019624,6.223781
inv_return_table(),0.018503,5.868249
pv_claims(kind),0.018216,5.776984
"claims(t, kind)",0.018125,5.748327
"av_at(t, timing)",0.016349,5.185010
prem_to_av_pp(t),0.015626,4.955751
pols_new_biz(t),0.015588,4.943653


In [17]:
df = pd.concat([df1[['duration']], df4[['duration']]], axis=1)
df.columns = ['df1', 'df4']
df.loc[(df['df1'] > 0.05) | (df['df4'] > 0.05)]

,df1,df4
surr_charge_rate(t),0.697034,0.024496
inv_return_mth(t),0.338032,0.000000
premium_pp(t),0.146663,0.033878
pols_new_biz(t),0.069975,0.015588
inv_income_pp(t),0.069556,0.015507
"claims(t, kind)",0.064405,0.018125
